# **[ Vecotorizer ]** for perfume recommendation system

## Vectorizer 시작

### Read, read!!

In [1]:
item_df = pd.read_csv("item_df_final_final.csv")

In [12]:
from ast import literal_eval
item_df["notes"] = item_df["notes"].apply(literal_eval)

countvectorize를 적용하기 위해 모든 노트들 + gender 를 하나의 문장으로 만들어주기

In [31]:
item_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5470 entries, 0 to 5469
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   item_url       5470 non-null   object 
 1   item_id        5470 non-null   object 
 2   review         5470 non-null   float64
 3   names          5470 non-null   object 
 4   notes          5470 non-null   object 
 5   house          5470 non-null   object 
 6   rating         5470 non-null   float64
 7   gender         5470 non-null   object 
 8   notes_literal  5470 non-null   object 
dtypes: float64(2), object(7)
memory usage: 384.7+ KB


In [32]:
item_df["notes_literal"] = item_df.notes.apply(lambda x: " ".join(x)) + " " + item_df.gender

In [15]:
import matplotlib.pyplot as plt
%matplotlib inline

**< 왜 countvectorize인가? >**

- TF-IDF를 적용할 경우, 많은 향수에 들어가있는 note에 역가중치가 부여될 것이기 때문에

In [20]:
# Notes 컬럼에 CountVectorizer 수행, 유사도 비교를 위해서?
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
note_mat = count_vect.fit_transform(item_df['notes_literal'])
print(note_mat.shape)

(5470, 24812)


=> 5470의 향수에 대한 24812개의 단어들 count

In [21]:
# 문장 유사도 측정 방법의 하나인 코사인 유사도 측정 수행!!
from sklearn.metrics.pairwise import cosine_similarity

note_sim = cosine_similarity(note_mat, note_mat)
print(note_sim.shape)
print(note_sim[:2])

(5470, 5470)
[[1.         0.09677419 0.13340747 ... 0.03745029 0.         0.32128773]
 [0.09677419 1.         0.13340747 ... 0.07490059 0.         0.08032193]]


In [49]:
# 카운터벡터라이즈로 빈칸에는 0을 넣고 거리를 비교해서,, confusion_matrix와 비슷하게 해석하면 된다.
#note_sim 객체에서 높은 값 순으로 정렬!

note_sim_sorted_ind = note_sim.argsort()[:, ::-1] 
# ::-1 은 역순으로 정렬해 주는 기능! argsort에는 ascending이나, descending기능이 따로 없음..
print(note_sim_sorted_ind[:1])

[[5469 5468 5467 ...  452  451    0]]


In [27]:
def find_sim_perfume(df, sorted_ind, perfume_name, top_n=50):
    title_perfume = df[df['names'] == perfume_name]
    
    title_index = title_perfume.index.values
    similar_indexes = sorted_ind[title_index, :(top_n)]
    
    print(similar_indexes)
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [35]:
# Aventus 비슷한 향수 찾아보기?

similar_perfumes = find_sim_perfume(item_df, note_sim_sorted_ind, 'Aventus', 50)
similar_perfumes_df = similar_perfumes[['names', 'rating', 'house', 'review', "item_url" ]].iloc[1:,:].sort_values(by="rating", ascending=False)
similar_perfumes_df

[[3441 2215 1493 3442 3016 4128 1492  107 2958  854 4545 5052 1085  748
     3 1531 3592 5247 3083  113 4067   31 4770 4924 1888 4465  315 5189
  3494 2592 1995 2079 3491 4285 2128 1929 5184 4768 5046 3088 2547 5404
  3003 5153 3498 1890 4426 2070 1120 5340]]


,names,rating,house,review,item_url
4768,Enigma pour Homme / Creation-E pour Homme,4.26,Wasserstein Perella,17.0,https://www.basenotes.net/ID26140237.html
4465,Diorella,4.12,Moet Hennessy,62.0,https://www.basenotes.net/ID10210708.html
3498,Balenciaga pour Homme,4.11,Bogart Group,48.0,https://www.basenotes.net/ID26120160.html
1531,Polo Crest,4.05,L'Oréal Group,36.0,https://www.basenotes.net/ID26121306.html
1929,Santal Noble,4.03,Shiseido,60.0,https://www.basenotes.net/ID26121442.html
113,Jasmin Impératrice Eugénie,4.02,Creed,29.0,https://www.basenotes.net/ID10210909.html
2215,Supremacy Silver,4.00,Lush,5.0,https://www.basenotes.net/ID26144140.html
4128,Club de Nuit Intense for Men,3.94,L'Oréal Group,36.0,https://www.basenotes.net/ID26148985.html
1493,Pineapple Vintage Intense,3.91,Freedom Marketing Group,5.0,https://www.basenotes.net/ID26152536.html
2592,Vermeil for Men,3.87,Estee Lauder Companies,27.0,https://www.basenotes.net/ID26122150.html
